# Descenso al fondo de un cráter en Marte 

- Diseño de agentes inteligentes
- TC2032.101
- Profesor: Juan Emanuel Ledesma Martínez
- Equipo 7:

| Alumno | Matrícula |
| ---- | ---- |
| Juan Pablo Echeagaray González | A00830646 |
| Emily Rebeca Méndez Cruz | A00830768 |
| Erika Martínez Meneses | A01028621 |
| Oscar Antonio Banderas Álvarez | A01568492 |
| César Guillermo Vázquez Alvarez | A01197857 |

Fecha: 13 de marzo del 2022

## Entregable

Se teoriza que en el fondo de algunos de los cráteres de la superficie marciana es posible encontrar agua y otros elementos interesantes que podrían responder a la pregunta de si en Marte alguna vez hubo vida o si la hay en estos momentos. Por ello, es interesante ver la posibilidad de que alguno de los Rover exploradores logre navegar de manera segura hasta estos cráteres y llegue al fondo de los mismos.

Considere que hay un grupo de ingenieros que desean determinar el tipo de explorador que podría llevar a cabo la tarea de descender por un cráter de manera exitosa. Para ello, quieren realizar una serie de simulaciones utilizando como ejemplo el siguiente cráter de la superficie marciana extraído del sitio de [HiRISE](https://www.uahirise.org/dtm/).

![](img/2022-03-14-15-02-24.png)

![](img/2022-03-14-15-02-38.png)

![](img/2022-03-14-15-02-51.png)

La información mostrada con anterioridad se puede representar con una imagen de falso color como la que se muestra a continuación. El color de cada pixel de la imagen representa la altura en dicha posición.

![](img/2022-03-14-15-03-12.png)

Los datos de este cráter tienen dimensiones de 7163 pixeles x 10770 pixeles. Cada pixel representa un área de 1.0045 metros x 1.0045 metros. Los datos mostrados representan la profundidad medida con instrumentos satelitales para cada pixel. El nivel más bajo corresponde a los 0 metros, y se representa con rojo. Los datos mostrados se pueden descargar aquí [Descargar aquí](https://experiencia21.tec.mx/courses/241470/files/78461918?wrap=1) . Para escalar los datos y generar una matriz de numpy utilicen este script [Descargar script](https://experiencia21.tec.mx/courses/241470/files/78461984/download?download_frd=1) . En los datos escalados, cada píxel representa un área de 10.045 metros x 10.045 metros.

El robot explorador que se desea enviar sólo es capaz de utilizar información de su alrededor para tomar decisiones. Es decir, sólo conocerá la profundidad en su vecindad. Por ello, los ingenieros decidieron que en la simulaciones, si el explorador se encuentra en cierto pixel de la imagen, sólo será capaz de conocer la profundidad en los ocho pixeles vecinos. Además, para evitar daños en el robot, no se permite que el explorador se mueva a pixeles con una diferencia de altura mayor a 0.5 metros de la posición en donde se encuentre.

Para llevar a cabo las simulaciones del descenso, como una primera aproximación, se desea probar algoritmos de búsqueda local tales como búsqueda codiciosa y recocido simulado como mecanismos de toma de decisiones del robot. Para ello, realicen lo siguiente.

### Búsqueda codiciosa

Implementen un algoritmo de búsqueda codiciosa o voraz, con el cual, dado una posición inicial en la imagen del cráter, el explorador busque un camino de descenso para llegar a la parte del cráter más profunda. Una manera de implementar dicho algoritmo sería la siguiente:

- Dado el pixel donde se encuentre el explorador, verifique las profundidades de los ocho pixeles vecinos de dicha posición.
- Escoja el pixel que tenga la menor profundidad posible, pero que la diferencia entre la altura actual y la de la posición vecina no sea mayor a 2.0 metros. Si esta profundidad es mejor que la actual, mueva el explorador a dicha posición.
- Repita lo mismo hasta que el explorador no sea capaz de encontrar una mejor posición.

Prueben su algoritmo en la posición x = 3350 metros y = 5800 metros. ¿Hasta donde es capaz el explorador de llegar? Prueben su algoritmo con otras cinco posiciones cercanas y lejanas al fondo del cráter. ¿Qué tan bueno es el algoritmo para llegar al fondo del cráter es los casos probados?

### Búsqueda con recocido simulado

Utilicen el algoritmo de recocido simulado para encontrar una ruta que lleve al fondo del cráter con el explorador. Cada iteración del algoritmo la pueden implementar de la siguiente manera:

- Dado el pixel donde se encuentre el explorador, seleccione uno de los pixeles vecinos de manera aleatoria, con la condición que la diferencia de alturas entre la posición actual y la del vecino no sea mayor a 0.5 metros.
- Si la nueva posición es mejor que la anterior, mueva el explorador a la posición anterior. De lo contrario, pruebe con la regla de recocido simulado si se acepta dicha nueva posición

Realicen las mismas pruebas llevadas a cabo búsqueda codiciosa. ¿Qué algoritmo logra llegar más profundo en el cráter? ¿Recomendarían a los ingenieros del robot utilizar alguno de estos algoritmos?

Al final, cada integrante del equipo deberá incluir sus contribuciones y conclusiones.

### Dependencias

In [14]:
from simpleai.search import SearchProblem
from simpleai.search import greedy, simulated_annealing
import numpy as np

from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Cascadia Code; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")


### Condiciones iniciales

In [15]:
SCALE = 10.0174
crater_map = np.load('crater_map.npy')
NR, NC = crater_map.shape

# Start position
x0, y0 = 3350, 5800
MAX_DIFF = 0.5
UNTRAVERSABLE = -1
STEP_SIZE = 1


### Funciones auxiliares

In [16]:
def pos_to_index(pos: list[int]) -> list[int]:
    nr = NR - round(pos[0] / SCALE)
    nc = round(pos[1] / SCALE)
    return [nr, nc]


def coord_to_state(coord: list[int]) -> str:
    return f'{coord[0]},{coord[1]}'


def state_to_coord(state: str) -> list[int]:
    return tuple([int(x) for x in state.split(',')])


def report(result):
    print(f'Found path with length {len(result.path())}')
    print(f'Path: {result.path()}')
    print(f'Cost: {result.cost}')


def distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)


def get_indices_of_k_smallest(arr, k):
    # https://stackoverflow.com/a/51523226/15180258
    idx = np.argpartition(arr.ravel(), k)
    
    return np.array(np.unravel_index(idx, arr.shape))[:, range(k)].transpose().tolist()


def check_surroundings(map, pos, delta):
    x, y = pos
    surroundings = map[x - delta:x + delta + 1, y - delta:y + delta + 1]
    return np.round(surroundings, decimals=2)


## Punto más bajo

Una aproximación a la solución de este probolema sería que nosotros mismos encontremos el punto más bajo del cráter, con esta información reduciríamos el problema de optimización a un problema de búsqueda. Para esta entrega pensamos usar la altura de este punto como un valor de referencia, con el cuál podremos medir el desempeño general de las implementaciones que hagamos.


In [17]:
x_min, y_min = np.unravel_index(
    np.argmin(np.sqrt(crater_map * crater_map), axis=None), crater_map.shape)
print(f'Min height: {crater_map[x_min, y_min]} at ({x_min}, {y_min})')


Min height: 0.09667470703125218 at (58, 66)


Hay que destacar que las _coordenadas_ que regresa la directiva anterior son índices de la matriz, no coordenadas $(x, y)$ de verdad.

### Analizando los alrededores

In [18]:
change = 4
check_surroundings(crater_map, (x_min, y_min), change)


array([[-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,  1.0000e-01,
         9.9800e+00,  1.9840e+01,  2.6380e+01,  3.4020e+01],
       [ 9.9400e+00,  3.7390e+01,  6.7060e+01,  9.4540e+01,  1.0859e+02,
         1.0834e+02,  1.0829e+02,  1.0819e+02,  1.0812e+02],
       [ 1.0838e+02,  1.0867e+02,  1.0874e+02,  1.0850e+02,  1.0807e+02,
         1.0774e+02,  1.0758e+02,  1.0751e+02,  1.0748e+02],
       [ 1.0792e+02,  1.0860e+02,  1.0933e+02,  1.0920e+02,  1

El punto asociado con la menor altura será inalcanzable para el agente si es que aplicamos la condición de que la máxima diferenccia de alturas sea de 0.5 metros.

In [19]:
k = 10
small_idx = get_indices_of_k_smallest(np.sqrt(crater_map * crater_map), k)
heights = [crater_map[x, y] for x, y in small_idx]


In [20]:
print(f'Top {k} Smallest heights')
for idx in small_idx:
    print(f'{crater_map[idx[0], idx[1]]} at ({idx[0]}, {idx[1]})')

# La salida de esta directiva no estará ordenada en un orden particular

Top 10 Smallest heights
0.36616933593750217 at (213, 9)
0.3613402343750022 at (24, 588)
0.35244375000000217 at (7, 504)
0.3410302734377183 at (583, 295)
0.2324828125000022 at (69, 21)
0.09667470703125218 at (58, 66)
0.26782226562521827 at (584, 294)
0.2879808593750022 at (70, 17)
0.1605760742187522 at (53, 130)
0.36660390625000217 at (48, 592)


In [21]:
for idx in small_idx:
    print(f'''
{crater_map[idx[0], idx[1]]} at ({idx[0]}, {idx[1]})
{check_surroundings(crater_map, idx, 3)}
----------------------------------------''')



0.36616933593750217 at (213, 9)
[[ -1.    -1.    -1.    31.63 134.41 133.86 133.79]
 [ -1.    -1.    -1.    23.52 135.12 134.81 134.42]
 [ -1.    -1.    -1.    12.65 135.46 135.33 135.06]
 [ -1.    -1.    -1.     0.37 135.65 135.48 135.54]
 [ -1.    -1.    -1.    -1.   128.93 135.68 135.91]
 [ -1.    -1.    -1.    -1.   119.33 135.73 135.81]
 [ -1.    -1.    -1.    -1.   100.26 135.81 135.77]]
----------------------------------------

0.3613402343750022 at (24, 588)
[[135.59 135.55  60.44  -1.    -1.    -1.    -1.  ]
 [135.91 135.72  90.47  -1.    -1.    -1.    -1.  ]
 [136.14 135.41 113.28  -1.    -1.    -1.    -1.  ]
 [135.29 134.83 131.87   0.36  -1.    -1.    -1.  ]
 [134.69 134.48 134.26  19.31  -1.    -1.    -1.  ]
 [134.27 133.86 134.29  40.96  -1.    -1.    -1.  ]
 [133.43 134.06 134.37  61.4   -1.    -1.    -1.  ]]
----------------------------------------

0.35244375000000217 at (7, 504)
[[ -1.    -1.    -1.    -1.    -1.    -1.    -1.  ]
 [ -1.    -1.    -1.    -1.    -1.   

## Implementación

In [92]:
class mars_descent(SearchProblem):

    def __init__(self, initial_state: str, mars_map: np.array, max_height: float, not_traversable: int, step_size: int):

        self.mars_map = mars_map
        self.initial_state = initial_state
        self.max_height = max_height
        self.not_traversable = not_traversable
        self.step_size = step_size
        print("Initial state: ", self.initial_state)

        SearchProblem.__init__(self, initial_state)

    def actions(self, state: str) -> list[str]:
        mars_map = self.mars_map
        actions = []

        pos_directions = ['U', 'D', 'L', 'R', 'UL', 'UR', 'DL', 'DR']
        x0, y0 = state_to_coord(state)
        current_height = mars_map[x0, y0]

        for action in pos_directions:
            x, y = state_to_coord(self.result(state, action))
            next_height = mars_map[x, y]
            height_difference = abs(current_height - next_height)

            if next_height != self.not_traversable:
                if height_difference <= self.max_height:

                    actions.append(action)

        return actions

    def result(self, state: str, action: str) -> str:
        x, y = state_to_coord(state)
        step_size = self.step_size

        if action.count('U'):
            x -= step_size
        elif action.count('D'):
            x += step_size
        elif action.count('L'):
            y -= step_size
        elif action.count('R'):
            y += step_size
        elif action.count('UL'):
            x -= step_size
            y -= step_size
        elif action.count('UR'):
            x -= step_size
            y += step_size
        elif action.count('DL'):
            x += step_size
            y -= step_size
        elif action.count('DR'):
            x += step_size
            y += step_size
        else:
            print('No se encontró la acción')

        new_state = coord_to_state([x, y])
        # print(f'Action: {action}, New position: {state}, New height: {self.mars_map[x, y]}', file=log)
        return new_state

    def is_goal(self, state: str) -> bool:
        # Esta función no tiene una definición como tal ya que el algoritmo no puede saber de antemano cual es el estado meta
        pass

    def value(self, state: str) -> float:
        # Los algoritmos de búsqueda local de simpleai tratan de maximizar el valor de value, como nosotros queremos minimizar el valor
        # de la altura, podemos calcular su recíporo, mientras menor sea la altura, mayor será su recíproco

        return 1 / self.mars_map[state_to_coord(state)]


In [23]:
START_STATE = coord_to_state(pos_to_index([x0, y0]))


In [24]:
crater_problem = mars_descent(
    START_STATE, crater_map, 50 * MAX_DIFF, UNTRAVERSABLE, STEP_SIZE)


Initial state:  743,579


In [25]:
# Log all print statements to a file
log = open('greedy-output.txt', 'w')


### Búsqueda voraz

In [26]:
greedy_res = greedy(crater_problem, graph_search=True)


KeyboardInterrupt: 

El método de búsqueda voraz es al final del día un método de búsqueda, no está enfocado en un proceso de optimización como su contraparte (_recocido simulado_). Las directivas de abajo podrían ser usadas una vez que el algoritmo encuentre el punto que tenga la menor altura, predecir qué tanto tiempo le tomará al algoritmo encontrar una solución es algo complicado, al momento de redactar esto lleva 2 horas corriendo.

Información sobre las acciones que está tomando el algoritmo pueden ser encontradas en el archivo `output.txt`. Este contendrá información acerca de los nodos que está visitando, es a través de este archivo que pudimos confirmar que el agente no está ciclado en sus alrededores, sino que sí está explorando el mapa en su totalidad.

Una segunda aproximación a este problema de optimización sería pasarle al algoritmo una función `is_goal` que solamente arroje valores `True` cuando este haya encontrado algún punto con una altura menor a un valor predeterminado, como equipo hemos implementado esta situación, y el algoritmo fue capaz de encontrar una solución en tiempos no mayores a 5 minutos; sin embargo, consideramos que proporcionar un valor _meta_ le quitaría la etiqueta de optimización al problema. Si quisiéramos aplicar este algoritmo a una situación real, no sabríamos de antemano a qué valor aspirar, solamente querremos que el algoritmo optimice una métrica.


In [ ]:
greedy_res.path()


In [ ]:
greedy_res.cost


In [ ]:
greedy_res.path()[-1][1]


In [ ]:
crater_map[state_to_coord(greedy_res.path()[-1][1])]


In [36]:
log.close()


### Recocido simulado

Para que el recocido simulado pueda encontrar un mínimo para la altura, debemos de darle la capacidad de que avance varias _casillas_. Si dejamos que el tamaño de paso se quede como 1, el recocido simulado no encontrará una solución en tiempos óptimos (nuestro equipo de cómputo no es tan avanzado como para soportar la carga computacional).

In [93]:
log = open('output.txt', 'w')


In [94]:
crater_problem = mars_descent(START_STATE, crater_map, 100000 * MAX_DIFF, UNTRAVERSABLE, 100 * STEP_SIZE)


Initial state:  743,579


In [95]:
ann_res = simulated_annealing(crater_problem)


In [96]:
ann_res.path()


[('U', '643,579')]

In [97]:
ann_res.cost


1

In [98]:
ann_res.value


0.0065950245627632255

In [99]:
crater_map[state_to_coord(ann_res.path()[-1][1])]


151.62945800781273

In [33]:
log.close()


## Analizando otros puntos

Como puntos iniciales de prueba proponemos:

1. $(100, 100)$
2. $(200, 315)$
3. $(500, 500)$
4. $(100, 600)$
5. $(89, 314)$

Para seleccionar estos puntos verificamos de antemano que no tuvieran una altura asociada de -1, una vez que el agente no se encuentre en uno de esos puntos podrá operar sin mayor problema.

In [101]:
# Posición inicial 743, 579
# Posibles posiciones de prueba [100, 100], [200, 315], [500, 500],  [100, 600], [89, 314]
test_positions = [[100, 100], [200, 315], [500, 500], [100, 600], [89, 314]]


### Probando a búsqueda voraz

In [107]:
for pos in test_positions:
    test_problem = mars_descent(coord_to_state(pos_to_index(pos)), crater_map, 100000 * MAX_DIFF, UNTRAVERSABLE, 5 * STEP_SIZE)
    greedy_res = greedy(test_problem, graph_search=True)
    print(f'Posición inicial: {pos}')
    print(f'Posición inicial: {pos}')
    print(f'Altura inicial: {crater_map[pos[0], pos[1]]}')
    print(f'Resultado: {greedy_res.path()}')
    print(f'Costo: {greedy_res.cost}')
    print(f'Valor: {greedy_res.value}')
    print(f'Altura: {crater_map[state_to_coord(greedy_res.path()[0][1])]}')
    print('----------------------------------------')
    

Initial state:  1067,10
Posición inicial: [100, 100]
Posición inicial: [100, 100]
Altura inicial: 122.71356445312522


AttributeError: 'NoneType' object has no attribute 'path'

### Probando a recocido simulado

In [104]:
for pos in test_positions:
    test_problem = mars_descent(coord_to_state(pos), crater_map, 100000 * MAX_DIFF, UNTRAVERSABLE, 100 * STEP_SIZE)
    annealing_res = simulated_annealing(test_problem)
    print(f'Posición inicial: {pos}')
    print(f'Altura inicial: {crater_map[pos[0], pos[1]]}')
    print(f'Resultado: {annealing_res.path()}')
    print(f'Costo: {annealing_res.cost}')
    print(f'Valor: {annealing_res.value}')
    print(f'Altura: {crater_map[state_to_coord(annealing_res.path()[0][1])]}')
    print('----------------------------------------')

Initial state:  100,100
Posición inicial: [100, 100]
Altura inicial: 122.71356445312522
Resultado: [('UL', '100,100')]
Costo: 2
Valor: 0.00814905837391746
Altura: 122.71356445312522
----------------------------------------
Initial state:  200,315
Posición inicial: [200, 315]
Altura inicial: 138.58318847656273
Resultado: [('UL', '100,315')]
Costo: 1
Valor: 0.007212835381436947
Altura: 138.6417333984377
----------------------------------------
Initial state:  500,500
Posición inicial: [500, 500]
Altura inicial: 153.74771728515645
Resultado: [('DL', '300,500')]
Costo: 4
Valor: 0.006897247232180646
Altura: 144.98537841796897
----------------------------------------
Initial state:  100,600
Posición inicial: [100, 600]
Altura inicial: 86.88685654296889
Resultado: [('DL', '200,600')]
Costo: 1
Valor: 0.006952251677485305
Altura: 143.83829101562523
----------------------------------------
Initial state:  89,314
Posición inicial: [89, 314]
Altura inicial: 136.80536865234396
Resultado: [('DR', '1

## Contribuciones

- Juan Pablo Echeagaray González: implementación, pruebas, documentación

## Conclusiones

- Juan Pablo Echeagaray González:
  - La tareas de optimización son en verdad complicadas, conceptualmente son fáciles de describir, su implementación en un lenguaje de programación también es relativamente sencilla, pero el proceso de optimización es realmente complicado para los algoritmos seleccionados cuando el poder de cómputo es muy bajo. Hay otras opciones de algoritmos de optimización que debemos de considerar, y como nota para investigaciones futuras, es importante priorizar que el equipo de cómputo a analizar tenga las capacidades suficientes para realizar estas tareas en un tiempo óptimo.